In [2]:
%pip install confluent-kafka
%pip install pyspark
%pip install pymongo
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import confluent_kafka as ck
import random
import time
import requests
import csv
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta, date
import calendar
import re
import time

# Kafka

## Config Kafka

In [9]:
config = {}
# Required connection configs for Kafka producer, consumer, and admin
config["bootstrap.servers"] = "pkc-60py3.europe-west9.gcp.confluent.cloud:9092"
#bootstrap.servers=pkc-60py3.europe-west9.gcp.confluent.cloud:9092
config["security.protocol"] = "SASL_SSL"
#security.protocol=SASL_SSL
config["sasl.mechanisms"] = "PLAIN"
#sasl.mechanisms=PLAIN
config["sasl.username"] = "2WQXJOQPDG2MWLJB"
#sasl.username=2WQXJOQPDG2MWLJB
config["sasl.password"] = "N0wpBXjhU3cBEawVm5aX5Ql5IofNTvWf4+CEHQgz+F2sinER5J/p6OJZpFJs0Hd5"
#sasl.password=N0wpBXjhU3cBEawVm5aX5Ql5IofNTvWf4+CEHQgz+F2sinER5J/p6OJZpFJs0Hd5

# Best practice for higher availability in librdkafka clients prior to 1.7
config["session.timeout.ms"] = "45000"
#session.timeout.ms=45000

config["group.id"] = "python-group-1"
config["auto.offset.reset"] = "earliest"
"""

# Required connection configs for Confluent Cloud Schema Registry
schema.registry.url=https://psrc-j55zm.us-central1.gcp.confluent.cloud
basic.auth.credentials.source=USER_INFO
basic.auth.user.info={{ SR_API_KEY }}:{{ SR_API_SECRET }}
"""

'\n\n# Required connection configs for Confluent Cloud Schema Registry\nschema.registry.url=https://psrc-j55zm.us-central1.gcp.confluent.cloud\nbasic.auth.credentials.source=USER_INFO\nbasic.auth.user.info={{ SR_API_KEY }}:{{ SR_API_SECRET }}\n'

# Consumer

In [14]:
from io import StringIO
consumer = ck.Consumer(config)
consumer.subscribe(["mon_topic_demo"])
df = pd.DataFrame(columns=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])
try:
     while True:
          msg = consumer.poll(1.0)
          if msg is not None and msg.error() is None:
               input_strings = msg.value().decode('utf-8').split('\n')
               for input_string in input_strings:
                    new_data = pd.read_csv(StringIO(input_string), sep=';', names=df.columns)
                    df = pd.concat([df, new_data], ignore_index=True)
except KeyboardInterrupt:
     pass
finally:
     consumer.close()




TypeError: initial_value must be str or None, not list

In [ ]:
display(df)